In [1]:
from time import sleep
from __future__ import division
from web3 import Web3, HTTPProvider
import json

import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set_style("whitegrid")
seaborn.despine()
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

# Connect to the network
web3 = Web3(HTTPProvider('http://localhost:8545'))

# Get contract signature
with open('../ethereum/build/contracts/Market.json') as energy_file:    
    energy_json = json.load(energy_file)
contract_abi = energy_json['abi']
network_id = energy_json['networks'].keys()[-1]
contract_address = energy_json['networks'][network_id]['address']

# List accounts and set default account
accounts = web3.eth.accounts
web3.eth.defaultAccount = accounts[0]
contract = web3.eth.contract(contract_abi, contract_address)
energy_posted_event = contract.on('energy_posted_event', filter_params={'fromBlock': 'earliest'})
participant_at_clearing_request = contract.on('participant_at_clearing_request', filter_params={'fromBlock': 'earliest'})
market_cleared_event = contract.on('market_cleared_event', filter_params={'fromBlock': 'earliest'})
bill_sent_event = contract.on('bill_sent_event', filter_params={'fromBlock': 'earliest'})

In [2]:
print('Number of accounts ' + str(len(accounts)))

# Estimated gas
gas = contract.estimateGas().add_participant()
print('Estimated gas per participant added ' + str(gas) + ' ETH')

# Add all participants
for account in accounts:
    trans_hash = contract.transact({'from': account, 'to': contract_address, 'gas': 900000}).add_participant()

    
# Check how many participants have been added
print('Number of participants ' + str(contract.call().number_of_participant()))

Number of accounts 10
Estimated gas per participant added 92294 ETH
Number of participants 10


In [3]:
def wait_until_trans_receipt(trans_hash):
    status = None
    while status is None:
        sleep(0.2)
        status = web3.eth.getTransactionReceipt(trans_hash)

In [5]:
energy_balances = [100, 500, 600, 200, 300, -100, -100, -200, -25, -50]
print('Total energy balance ' + str(sum(energy_balances)) + ' wh')
assert(len(accounts) == len(energy_balances))

# Estimated gas
gas = contract.estimateGas().post_energy_balance(int(10))
print('Estimated gas per participation ' + str(gas) + ' ETH')

# Post consumption and production for each account
for account, energy_balance in zip(accounts, energy_balances):
    temp = {'from': account, 'to': contract_address, 'gas': 900000}
    trans_hash = contract.transact(temp).post_energy_balance(int(energy_balance))
    wait_until_trans_receipt(trans_hash)

# Estimated gas
gas = contract.estimateGas().clear_market()
print('Estimated gas to clear market ' + str(gas) + ' ETH')

# Clear the market
contract.transact({'to': contract_address, 'gas': 4000000}).clear_market()
wait_until_trans_receipt(trans_hash)

# time.sleep(3)

# # Estimated gas
# gas = contract.estimateGas().bill_all_participants()
# print('Estimated gas to bill participants ' + str(gas) + ' ETH')

# contract.transact({'to': contract_address, 'gas': 4000000}).bill_all_participants()

Total energy balance 1225 wh
Estimated gas per participation 68318 ETH
Estimated gas to clear market 87771 ETH


In [110]:
participant_at_clearing_request.get(only_changes=False)

[{'address': u'0xfc5303c96a7fc61c9b3fa0e368885c5aebb73652',
  'args': {u'_nbparticipation': 10, u'_totparticipant': 10},
  'blockHash': u'0x0a2997c912907ab59e5cae6fa950d75cdecc89851812916b05c7ed033db5e219',
  'blockNumber': 28,
  'event': u'participant_at_clearing_request',
  'logIndex': 0,
  'transactionHash': u'0x0cbcecf88ac42c6df932be3523917e41d8410d910141f4bc9ffab3003f5652db',
  'transactionIndex': 0},
 {'address': u'0xfc5303c96a7fc61c9b3fa0e368885c5aebb73652',
  'args': {u'_nbparticipation': 10, u'_totparticipant': 10},
  'blockHash': u'0x0f514a64494c713b1db429846b5dd9f45c23bcc7ced783b4d0164f6e41e2e125',
  'blockNumber': 39,
  'event': u'participant_at_clearing_request',
  'logIndex': 0,
  'transactionHash': u'0x44b101b5c154cabc7d1ab5d0cfffb76379e9621794d795b846de564c95d4e8a1',
  'transactionIndex': 0},
 {'address': u'0xfc5303c96a7fc61c9b3fa0e368885c5aebb73652',
  'args': {u'_nbparticipation': 10, u'_totparticipant': 10},
  'blockHash': u'0x5d35094672cc44452386ce423d8e936845310ae5

In [78]:
market_cleared_event.get(only_changes=False)

[{'address': u'0x5e599d37de8bfbc1fb7584debf9ec39b376447a4',
  'args': {u'_buy': 105,
   u'_gen': 475,
   u'_prod': 1700,
   u'_sell': 120,
   u'ratio': 357},
  'blockHash': u'0xa0f451c5288eb95f82e0bd767a0785d2cb812d3a6213baa4c840b812c4909b67',
  'blockNumber': 28,
  'event': u'market_cleared_event',
  'logIndex': 1,
  'transactionHash': u'0x1753c9c1fdf3cdcc66bd6ffb1c7c9100a8633bcc39ca9162214b1aff1a6b21ad',
  'transactionIndex': 0}]

In [79]:
energy_posted_event.get(only_changes=False)

[{'address': u'0x5e599d37de8bfbc1fb7584debf9ec39b376447a4',
  'args': {u'_target': u'0xa0cddae21dc9bdd024f023e857ebac9b00a36d1f',
   u'_value': 100},
  'blockHash': u'0x4f296eeaf104f29c5fd6fdf2b6051fe20a8f0e0dda352eae7162b33fccc188ea',
  'blockNumber': 18,
  'event': u'energy_posted_event',
  'logIndex': 0,
  'transactionHash': u'0x81548af6e5719e58325ccb3fae47361177e44dc22e460e0d1bd486c54b5e28cc',
  'transactionIndex': 0},
 {'address': u'0x5e599d37de8bfbc1fb7584debf9ec39b376447a4',
  'args': {u'_target': u'0x9d442647c69f925cf9c7162fc2d431a1026d230c',
   u'_value': 500},
  'blockHash': u'0xfb1d04e5814e43f9d8bd285234398fb4c7d9b809670c94bc02ac1f0691b37b19',
  'blockNumber': 19,
  'event': u'energy_posted_event',
  'logIndex': 0,
  'transactionHash': u'0x2e529c09f555427067d83213b99db0de0f46d40e6eeee092ad663311d119aa03',
  'transactionIndex': 0},
 {'address': u'0x5e599d37de8bfbc1fb7584debf9ec39b376447a4',
  'args': {u'_target': u'0x3780e31b33ca1c553de442da52c74f9f7c824447',
   u'_value': 60

In [80]:
bill_sent_event.get(only_changes=False)

[{'address': u'0x5e599d37de8bfbc1fb7584debf9ec39b376447a4',
  'args': {u'_target': u'0xa0cddae21dc9bdd024f023e857ebac9b00a36d1f',
   u'_value': 10500},
  'blockHash': u'0xa0f451c5288eb95f82e0bd767a0785d2cb812d3a6213baa4c840b812c4909b67',
  'blockNumber': 28,
  'event': u'bill_sent_event',
  'logIndex': 2,
  'transactionHash': u'0x1753c9c1fdf3cdcc66bd6ffb1c7c9100a8633bcc39ca9162214b1aff1a6b21ad',
  'transactionIndex': 0},
 {'address': u'0x5e599d37de8bfbc1fb7584debf9ec39b376447a4',
  'args': {u'_target': u'0x9d442647c69f925cf9c7162fc2d431a1026d230c',
   u'_value': 52500},
  'blockHash': u'0xa0f451c5288eb95f82e0bd767a0785d2cb812d3a6213baa4c840b812c4909b67',
  'blockNumber': 28,
  'event': u'bill_sent_event',
  'logIndex': 3,
  'transactionHash': u'0x1753c9c1fdf3cdcc66bd6ffb1c7c9100a8633bcc39ca9162214b1aff1a6b21ad',
  'transactionIndex': 0},
 {'address': u'0x5e599d37de8bfbc1fb7584debf9ec39b376447a4',
  'args': {u'_target': u'0x3780e31b33ca1c553de442da52c74f9f7c824447',
   u'_value': 63000}